In [20]:
import numpy as np
import skfuzzy as fuzz
from skfuzzy import control as ctrl
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score


## Modélisation floue

In [ ]:
temperature = ctrl.Antecedent(np.arange(0, 101, 1), 'temperature')
vibration = ctrl.Antecedent(np.arange(0, 11, 1), 'vibration')
age = ctrl.Antecedent(np.arange(0, 21, 1), 'age')

risque = ctrl.Consequent(np.arange(0, 11, 1), 'risque')

In [ ]:
temperature['basse'] = fuzz.trimf(temperature.universe, [0, 0, 40])
temperature['normale'] = fuzz.trimf(temperature.universe, [30, 50, 70])
temperature['elevee'] = fuzz.trimf(temperature.universe, [60, 100, 100])

vibration['faible'] = fuzz.trimf(vibration.universe, [0, 0, 4])
vibration['moyenne'] = fuzz.trimf(vibration.universe, [2, 5, 8])
vibration['forte'] = fuzz.trimf(vibration.universe, [6, 10, 10])

age['neuf'] = fuzz.trimf(age.universe, [0, 0, 7])
age['moyen'] = fuzz.trimf(age.universe, [5, 10, 15])
age['ancien'] = fuzz.trimf(age.universe, [12, 20, 20])

risque['faible'] = fuzz.trimf(risque.universe, [0, 0, 4])
risque['moyen'] = fuzz.trimf(risque.universe, [2, 5, 8])
risque['eleve'] = fuzz.trimf(risque.universe, [6, 10, 10])

In [23]:
rule1 = ctrl.Rule(temperature['elevee'] | vibration['forte'], risque['eleve'])
rule2 = ctrl.Rule(age['ancien'] & vibration['moyenne'], risque['moyen'])
rule3 = ctrl.Rule(temperature['basse'] & vibration['faible'] & age['neuf'], risque['faible'])
rule4 = ctrl.Rule(temperature['normale'] & age['moyen'], risque['moyen'])


In [24]:
risque_ctrl = ctrl.ControlSystem([rule1, rule2, rule3, rule4])
risque_sim = ctrl.ControlSystemSimulation(risque_ctrl)

# Exemple de simulation
risque_sim.input['temperature'] = 65
risque_sim.input['vibration'] = 8
risque_sim.input['age'] = 15

risque_sim.compute()
print("Risque de panne défuzzifié :", risque_sim.output['risque'])


Risque de panne défuzzifié : 8.444444444444445


## Machine Learning

In [ ]:
data = []

for i in range(2000):
    t = np.random.uniform(0, 100)
    v = np.random.uniform(0, 10)
    a = np.random.uniform(0, 20)

    sim = ctrl.ControlSystemSimulation(risque_ctrl)
    sim.input['temperature'] = t
    sim.input['vibration'] = v
    sim.input['age'] = a

    sim.compute()

    risque_val = sim.output.get('risque', np.nan)
    data.append([t, v, a, risque_val])

df = pd.DataFrame(data, columns=['temperature', 'vibration', 'age', 'risque'])
df.dropna(inplace=True)
df.head()


,temperature,vibration,age,risque
0,83.217299,3.676670,4.645536,8.501323
3,9.143783,3.821211,13.241068,5.000000
4,27.820868,7.950215,19.669136,8.235564
5,93.654390,7.352042,5.551729,8.637705
6,45.445496,9.956404,4.336043,8.666510


In [26]:
X = df[['temperature', 'vibration', 'age']]
y = df['risque']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

model = RandomForestRegressor(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

y_pred = model.predict(X_test)

print("MSE :", mean_squared_error(y_test, y_pred))
print("R² :", r2_score(y_test, y_pred))


MSE : 0.027435971438578552
R² : 0.9925932896377263
